In [10]:
# show working directory
import os

# Change working directory
os.chdir('c:\\Users\\Soren\\Documents\\GitHub\\IntroSocial23\\Exam final')

In [58]:
# Creating sentiment dataframe
import os
import glob
import pandas as pd
from datetime import datetime, timedelta

In [11]:
# Set the path to the folder containing the CSV files
Path_sentiment = os.getcwd()+'/Data collection/News data/Sentiment analysis'

# Set the path to the folder containing the news_final_1 CSV file
Path_news_final_1 = os.getcwd()+'/Data collection/News data'

# Importing the news_final_1 CSV file
news_final_1 = pd.read_csv(Path_news_final_1+'/news_final_1.csv', sep = ';', encoding = 'utf-8')

# List comprehension to find all CSV files in the folder
csv_files = [file for file in os.listdir(Path_sentiment) if file.endswith('.csv')]

# Store all the dataframes in one dataframe 
list_of_dfs = [pd.read_csv(os.path.join(Path_sentiment, csv_file)) for csv_file in csv_files]

# Concatenate all the dataframes in one dataframe
sentiment_df = pd.concat(list_of_dfs)

# Left joining dataframes on date and company
news_final_merge = pd.merge(sentiment_df, news_final_1,  how='left', on=['date', 'company'])

# drop duplicates that matches on all columns except for tickers to avoid duplicates 
news_clean = news_final_merge.drop_duplicates(subset=['LLM_score','date','company','title','symbols','sentiment'], keep='first')


In [12]:
# Replace typos in LLM_score
news_clean['LLM_score'] = news_clean['LLM_score'].replace('0 Ul', 0)
news_clean['LLM_score'] = news_clean['LLM_score'].replace('0 Ve', 0)

# convert LLM_score to integer
news_clean['LLM_score'] = news_clean['LLM_score'].astype('int32')

# Check the value counts of LLM_score
news_clean['LLM_score'].value_counts()

C:\Users\Soren\AppData\Local\Temp\ipykernel_24852\3362842792.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_clean['LLM_score'] = news_clean['LLM_score'].replace('0 Ul', 0)
C:\Users\Soren\AppData\Local\Temp\ipykernel_24852\3362842792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_clean['LLM_score'] = news_clean['LLM_score'].replace('0 Ve', 0)
C:\Users\Soren\AppData\Local\Temp\ipykernel_24852\3362842792.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

LLM_score
 0    90485
 1    55210
-1    31958
Name: count, dtype: int64

In [55]:
# Grouping by date and company and summing and taking the mean of the LLM_score
news_sent_final = news_clean.groupby(['date', 'company'])['LLM_score'].agg(['sum', 'mean'])

# Resetting the index
news_sent_final = news_sent_final.reset_index()
# news_sent_final.sort_values(by=['sum'], ascending=False)

news_sent_final['mean'].value_counts()

mean
 0.0    88966
 1.0    53924
-1.0    31525
 0.5      318
-0.5      100
Name: count, dtype: int64

In [57]:
news_sent_final

,date,company,sum,mean,Ticker
0,2022-06-01T00:53:45+00:00,Riot Platforms Inc,0,0.0,RIOT
1,2022-06-01T02:27:00+00:00,Forrester Research Inc,0,0.0,FORR
2,2022-06-01T03:50:00+00:00,Jack In The Box Inc,0,0.0,JACK
3,2022-06-01T03:57:00+00:00,Lyft Inc,0,0.0,LYFT
4,2022-06-01T04:00:30+00:00,Ambarella Inc,0,0.0,AMBA
...,...,...,...,...,...
174828,2023-06-01T23:44:33+00:00,Progress Software Corporation Common Stock (DE),-1,-1.0,PRGS
174829,2023-06-01T23:44:33+00:00,Rapid7 Inc,-1,-1.0,RPD
174830,2023-06-01T23:52:40+00:00,Catalyst Pharmaceuticals Inc,0,0.0,CPRX
174831,2023-06-01T23:52:40+00:00,Dynavax Technologies Corporation Common Stock,0,0.0,DVAX


In [61]:
# Importing the TickerNames_final CSV file
Path_Ticker = os.getcwd()+'/Data collection/'
TickerNames_final = pd.read_csv(Path_Ticker+'TickerNames_final.csv')

# Set 'Name' column as the index of TickerNames_final
TickerNames_final.set_index('Name', inplace=True)

# Create a dictionary to map company names to tickers using the index
ticker_mapping = TickerNames_final['Symbol'].to_dict()

# Apply the mapping to the 'company' column
news_sent_final['Ticker'] = news_sent_final['company'].map(ticker_mapping)

# Convert the timestamp column to datetime
news_sent_final['date'] = pd.to_datetime(news_sent_final['date'])

# Create a timedelta to represent the GMT-4 offset
gmt_minus_4 = timedelta(hours=-4)

# Define a function to apply the conversion
def convert_to_gmt_minus_4(timestamp):
    converted_datetime = timestamp + gmt_minus_4
    return converted_datetime.strftime("%Y-%m-%d")

# Apply the conversion function to the timestamp column
news_sent_final['New_date'] = news_sent_final['date'].apply(convert_to_gmt_minus_4)

# Change column names
news_sent_final = news_sent_final[['New_date', 'Ticker', 'sum', 'mean']]
colnames = ['date','Ticker','LLM_score_sum','LLM_score_mean']
news_sent_final.columns = colnames

news_sent_final


,date,Ticker,LLM_score_sum,LLM_score_mean
0,2022-05-31,RIOT,0,0.0
1,2022-05-31,FORR,0,0.0
2,2022-05-31,JACK,0,0.0
3,2022-05-31,LYFT,0,0.0
4,2022-06-01,AMBA,0,0.0
...,...,...,...,...
174828,2023-06-01,PRGS,-1,-1.0
174829,2023-06-01,RPD,-1,-1.0
174830,2023-06-01,CPRX,0,0.0
174831,2023-06-01,DVAX,0,0.0


In [63]:
# Save news_sent_final as a CSV file
news_sent_final.to_csv('NEWS_SENTIMENT_FINAL', sep = ';', encoding = 'utf-8')

In [76]:
# Read Master dataframe
Path_Master = os.getcwd()+'/Data processing/'

Master_df = pd.read_csv(Path_Master+'Master_with_technicals_news.csv', encoding='utf-8')

# Final dataframe
Final_df = pd.merge(Master_df, news_sent_final,  how='left', on = ['date', 'Ticker'])

# Count instances of LLM_score_sum
Final_df['LLM_score_mean'].value_counts()

# If LLM_score_mean is NaN, then it is set to 0 to indicate that there is no relevant news
Final_df['LLM_score_mean'] = Final_df['LLM_score_mean'].fillna(0)
Final_df['LLM_score_sum'] = Final_df['LLM_score_sum'].fillna(0)

# Save Final_df as a CSV file
Final_df.to_csv('FINAL_DF.csv', sep = ';', encoding = 'utf-8')



,date,open,high,low,adjusted_close,volume,RSI,MFI,volatility,returns,prev_returns,returns_pred,Ticker,LLM_score_sum,LLM_score_mean
0,2023-06-02,8.01,8.0900,7.8700,8.0600,74700,66.502362,48.430352,0.029908,2.025316,-1.250000,0.868486,AADI,0.0,0.0
1,2023-06-02,14.90,15.0700,14.6400,14.8700,31041300,23.119364,35.726831,0.014020,1.156463,-0.541272,-0.403497,AAL,0.0,0.0
2,2023-06-02,2.21,2.2700,2.1600,2.2700,148200,26.678504,47.236530,0.089979,2.714932,-0.450450,2.643172,AAOI,0.0,0.0
3,2023-06-02,88.82,91.6800,88.7300,91.5033,243389,36.479318,50.327996,0.016983,4.459876,1.223862,-0.458563,AAON,0.0,0.0
4,2023-06-02,181.03,181.7800,179.2600,180.7060,61945900,36.976220,45.237000,0.009638,0.477572,1.602217,-0.757141,AAPL,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561752,2022-06-01,33.26,33.3400,32.2600,32.9700,557900,68.832023,57.020542,0.033060,0.487656,-3.300914,3.639672,ZUMZ,-1.0,-1.0
561753,2022-06-01,4.64,4.7050,4.4900,4.5700,257036,45.655482,44.451853,0.035933,-1.295896,-1.279318,3.501094,ZVRA,0.0,0.0
561754,2022-06-01,6.85,7.1100,6.6450,6.9800,996500,59.232450,56.263023,0.052836,3.869048,-2.183406,-3.295129,ZYME,0.0,0.0
561755,2022-06-01,0.97,0.9899,0.9021,0.9050,237207,39.897928,87.930630,0.087510,-5.483029,-1.875384,7.182320,ZYNE,0.0,0.0
